##Factiva Results Parser

This notebook will allow you to parse results files downloaded from *Factiva*.  Specifically, I searched for *Wall St. Journal* articles on ~300 North American companies -- one at a time. For each company, I then selected all found articles ('headline' results) and emailed myself the results in HTML format.  I then saved each attached file with the relevant stock ticker name; for instance, *AAPL.html* might contain 37 articles for Apple. This notebook is designed to loop over all and parse all the individual HTML files and convert the output to an Excel spreadsheet containing one row per article, with columns for date, source, company, article headline, and article body.

<br>

In [3]:
import numpy as np
import pandas as pd

from pandas import DataFrame
from pandas import Series

from bs4 import BeautifulSoup
import time
import datetime
import re

#Set PANDAS to show all columns in DataFrame
pd.set_option('display.max_columns', None)

pd.set_option('max_colwidth',500)

print pd.__version__

0.16.2


### Set Working Directory

In [7]:
import os
indir = '/Factiva_2014'

### Import Excel File with List of Tickers 

I created a spreadsheet that included the names of the companies I was searching for. I also included a count of the number of articles I found on Factiva. I recommend doing the same in order to verify that the parser is working correctly. Let's read in the spreadsheet.

In [365]:
df = pd.read_excel('373 original tickers.xls', 'sheet1')
print len(df)
df.head(2)

373


,ticker,comnam,2013 WSJ ARTICLES,2014 WSJ ARTICLES,permco,permno,cusip,ncusip,screen_name,exclude,ticker_change_2013,PRIMEXCH,shared_account,Number_of_Twitter_Accounts,notes
5,ACO,AMCOL INTERNATIONAL CORP,NaN,NaN,9425,11599,02341W10,02341W10,none,exclude,NaN,NaN,NaN,NaN,merger in 2014
19,ARSD,ARABIAN AMERICAN DEVELOPMENT CO,NaN,NaN,352,14761,3846510,3846510,none,exclude,NaN,NaN,NaN,NaN,2014 coy and ticker name change -- replaced by $TREC


### Create Empty Dataframes for Articles and Article Counts

This first dataframe will just contain a count of the number of articles per company. This is not essential so skip if you'd like.

In [366]:
columns=['ticker', 'count']
counts = pd.DataFrame(data=np.zeros((0,len(columns))), columns=columns)
counts

,ticker,count


This is the key dataframe. It will contain one row per article with columns for ticker, date of article, article source, etc. 

In [367]:
#df_WSJ = pd.DataFrame(data, columns=['ticker', 'date', 'headline'])
columns=['ticker', 'count', 'date', 'source', 'headline', 'body']
df_WSJ = pd.DataFrame(data=np.zeros((0,len(columns))), columns=columns)
df_WSJ

,ticker,count,date,source,headline,body


### Loop Over Every HTML File in Directory, Parse Articles Table, Add Each Article to Dataframe

This loop does the heavy lifting. It will loop over every html file in the directory you've set above, and output the results to the article dataframe. 

In [383]:
for root, dirs, filenames in os.walk(indir):
    for f in filenames[1:]:     #NEED TO START AT 1 BECAUSE OF DS.STORE
        print f
        WSJ_ticker = f.replace('.html', '')   #TO GET THE TICKER NAME 
        #print WSJ_ticker
        file = indir+'/'+f
        #print file
        data = open(os.path.join(root, f), 'r').read()
        
        soup=BeautifulSoup(data)   #READ HTML FILE INTO BEAUTIFULSOUP
        #print soup.prettify()
        
        #print len(soup.findAll("table"))
        table = soup.findAll("table")[1]         #SELECT THE TABLE IN THE HTML FILE THAT CONTAINS THE ARTICLES
        
        #table = soup.find("table"), attrs={"style":"border-collapse"})
        #print table
 
        table_body = table.findAll('tbody')[2]   #SELECT THE THIRD 'TBODY' SECTION OF THE TABLE

        #print table_body
        #rows = table_body.find_all('tr')
        rows = table_body.find_all('td', {'class':'article'})   #SELECT ALL ROWS OF TABLE_BODY THAT CONTAIN ARTICLES
        print len(rows)
        
        counts = counts.append({"ticker": WSJ_ticker, 'count':len(rows)}, ignore_index=True)
        data = []       
        counter = 0
        for row in rows:
            #print row.text.strip()                       #THIS GETS ALL OF THE TEXT
            headline = row.find('h3').text.strip()        #GET ARTICLE HEADLINE; ASSIGN IT TO VARIABLE
            print headline    
            body = row.find('p', {'class':'articleBody'}).text.strip()  #GET ARTICLE BODY; ASSIGN IT TO VARIABLE
            #print body     
            meta = row.find('p', {'class':'articleMeta'}).text.strip()   #GET ARTICLE META DATA; ASSIGN IT TO VARIABLE
            #print meta
            meta_list = meta.split(',')                   #SPLIT META BY COMMAS INTO LIST
            
            source = meta_list[0]                         #SOURCE FOR THE ARTICLE (e.g., WSJ) IS THE FIRST ITEM IN LIST
            #print meta_list
            
            #FACTIVA RESULTS DO NOT INCLUDED A STANDARD SET OF ITEMS IN THE META DATA. I NEEDED TO FIND THE DATA, 
            #SO I PLAYED AROUND WITH THE FOLLOWING LOOP; USEFUL FOR SEARCHING FOR AN ITEM AND THEN GOING TO THE 
            #PREVIOUS OR NEXT ITEM IN THE LIST 
            #for index, obj in enumerate(meta_list):
            #    if 'GMT' in obj:
            #        #print obj, meta_list[index - 1]
            #        print meta_list[index - 1]
            #        #counter +=1
            
            #INSTEAD, THIS IS WHAT I DID. I USED A LIST COMPREHENSION OF ALL ITEMS THAT CONTAINED THE WORD '2014' AND 
            #THEN SELECTED THE FIRST ITEM. THIS RELIABLY GETS US THE ARTICLE DATE.
            date = [m for m in meta_list if '2014' in m][0]
            date = date.strip()  #REMOVE LEADING AND TRAILING SPACES
            print date
            
            converted_date = datetime.datetime.strptime(date, '%d %B %Y')  #CONVERT DATE TO PYTHON DATETIME OBJECT
            print converted_date
            
            df_WSJ = df_WSJ.append({"ticker": WSJ_ticker, 'count':len(rows), 'date':converted_date, 'source':source, 'headline':headline, 'body':body}, ignore_index=True)

ABX.html
58
Chile’s Top Court Won’t Hear Barrick Gold  Appeal; Company Appealed Lower-Court Ruling Related to Pascua-Lama Fines
31  December  2014
2014-12-31 00:00:00
Barrick to Suspend Operations at Zambia Copper Mine; Move Comes After Zambia Increases Royalty Rate
18  December  2014
2014-12-18 00:00:00
Corporate News: Barrick CFO to Depart
20  November  2014
2014-11-20 00:00:00
Firms Show Early Interest in Anglo American ’s Chilean Copper Assets; A Lack of Public Information Is Hindering Potential Bidders
07  November  2014
2014-11-07 00:00:00
Barrick May Close Zambian Copper Mine If Tax Hike Goes Ahead; Royalty Hike From 6% to 20% Will Make Production Uneconomic, Company Says
30  October  2014
2014-10-30 00:00:00
Barrick’s Earnings Decline on Lower Gold and Copper Prices; Miner Beats Street View, Helped by Efforts to Lower Mining Costs and Focus on Most Profitable Projects
29  October  2014
2014-10-29 00:00:00
Mining Force Mick Davis Digs In Again; With Xstrata  Veterans, He Looks t

<br>
Let's take a look at the main output.

In [369]:
print len(df_WSJ)
df_WSJ.head(2)

1786


,ticker,count,date,source,headline,body
0,ABX,58,2014-12-31,The Wall Street Journal Online,Chile’s Top Court Won’t Hear Barrick Gold Appeal; Company Appealed Lower-Court Ruling Related to Pascua-Lama Fines,"TORONTO—Barrick Gold Corp. said Wednesday that Chile’s top court won’t hear its appeal of a lower-court decision regarding sanctions imposed on the company’s Pascua-Lama project by the country’s environmental regulator, on the basis that ..."
1,ABX,58,2014-12-18,The Wall Street Journal Online,Barrick to Suspend Operations at Zambia Copper Mine; Move Comes After Zambia Increases Royalty Rate,Barrick Gold Corp. said Thursday it will suspend operations at a copper mine in Zambia and record an impairment charge after the country passed legislation to more than triple the royalty rate on open-pit mining operations.


And here's the supplementary 'counts' dataframe. The *count* variable here and in the above dataframe is the number of 'article rows' that were in each HTML file. We'll do another check below.

In [370]:
counts

,ticker,count
0,ABX,58
1,ACI,11
2,ACMP,5
3,AEM,15
4,AG,1
5,ANR,9
6,ANV,1
7,APA,22
8,APAGF,2
9,APC,28


<br>
Let's merge our company-level dataframe with our *counts* dataframe.

In [371]:
merged = pd.merge(df, counts, left_on='ticker', right_on='ticker', how='outer')
print len(merged)
merged

373


,ticker,comnam,2013 WSJ ARTICLES,2014 WSJ ARTICLES,permco,permno,cusip,ncusip,screen_name,exclude,ticker_change_2013,PRIMEXCH,shared_account,Number_of_Twitter_Accounts,notes,count
0,ACO,AMCOL INTERNATIONAL CORP,NaN,NaN,9425,11599,02341W10,02341W10,none,exclude,NaN,NaN,NaN,NaN,merger in 2014,NaN
1,ARSD,ARABIAN AMERICAN DEVELOPMENT CO,NaN,NaN,352,14761,3846510,3846510,none,exclude,NaN,NaN,NaN,NaN,2014 coy and ticker name change -- replaced by $TREC,NaN
2,AZC,AUGUSTA RESOURCE CORP,NaN,NaN,51410,91563,5091220,5091220,none,exclude,NaN,NaN,NaN,NaN,merger in 2014,NaN
3,AZK,AURIZON MINES LTD,NaN,NaN,44615,89903,05155P10,05155P10,none,exclude,NaN,NaN,NaN,NaN,merger in 2013,NaN
4,BRD,BRIGUS GOLD CORP,NaN,NaN,44264,89802,10949010,10949010,BrigusGold,exclude,NaN,NaN,NaN,NaN,merger in 2014,NaN
5,BRY,BERRY PETROLEUM CO,NaN,NaN,9309,11478,8578910,8578910,linnenergy,exclude,NaN,NaN,yes,NaN,merger in 2013,NaN
6,CDY,CARDERO RESOURCE CORP,NaN,NaN,46090,90519,14140U10,14140U10,none,exclude,NaN,NaN,NaN,NaN,dropped in 2013,NaN
7,CEP,CONSTELLATION ENERGY PTNRS L L C,NaN,NaN,51957,91845,210380000000000,210380000000000,ConstellationEG,exclude,NaN,NaN,NaN,NaN,2014 coy and ticker name change -- replaced by $SPP,NaN
8,CGR,CLAUDE RESOURCES INC,NaN,NaN,44704,89951,18287310,18287310,ClaudeResource,exclude,NaN,NaN,NaN,NaN,dropped in 2013,NaN
9,CMLP,CRESTWOOD MIDSTREAM PARTNERS L P,NaN,NaN,52701,92255,22637210,22637210,none,exclude,NaN,NaN,NaN,NaN,merger in 2013,NaN


<br>
Now let's also count how many articles there are per company in our output dataset. 

In [372]:
row_counts = df_WSJ.groupby('ticker').size()
row_counts = pd.DataFrame(row_counts, columns = ['num_articles'])
row_counts = row_counts.reset_index()
row_counts

,ticker,num_articles
0,ABX,58
1,ACI,11
2,ACMP,5
3,AEM,15
4,AG,1
5,ANR,9
6,ANV,1
7,APA,22
8,APAGF,2
9,APC,28


<br>
Now let's merge the row count with our merged dataframe.

In [373]:
merged = pd.merge(merged, row_counts, left_on='ticker', right_on='ticker', how='outer')
print len(merged)
merged

373


,ticker,comnam,2013 WSJ ARTICLES,2014 WSJ ARTICLES,permco,permno,cusip,ncusip,screen_name,exclude,ticker_change_2013,PRIMEXCH,shared_account,Number_of_Twitter_Accounts,notes,count,num_articles
0,ACO,AMCOL INTERNATIONAL CORP,NaN,NaN,9425,11599,02341W10,02341W10,none,exclude,NaN,NaN,NaN,NaN,merger in 2014,NaN,NaN
1,ARSD,ARABIAN AMERICAN DEVELOPMENT CO,NaN,NaN,352,14761,3846510,3846510,none,exclude,NaN,NaN,NaN,NaN,2014 coy and ticker name change -- replaced by $TREC,NaN,NaN
2,AZC,AUGUSTA RESOURCE CORP,NaN,NaN,51410,91563,5091220,5091220,none,exclude,NaN,NaN,NaN,NaN,merger in 2014,NaN,NaN
3,AZK,AURIZON MINES LTD,NaN,NaN,44615,89903,05155P10,05155P10,none,exclude,NaN,NaN,NaN,NaN,merger in 2013,NaN,NaN
4,BRD,BRIGUS GOLD CORP,NaN,NaN,44264,89802,10949010,10949010,BrigusGold,exclude,NaN,NaN,NaN,NaN,merger in 2014,NaN,NaN
5,BRY,BERRY PETROLEUM CO,NaN,NaN,9309,11478,8578910,8578910,linnenergy,exclude,NaN,NaN,yes,NaN,merger in 2013,NaN,NaN
6,CDY,CARDERO RESOURCE CORP,NaN,NaN,46090,90519,14140U10,14140U10,none,exclude,NaN,NaN,NaN,NaN,dropped in 2013,NaN,NaN
7,CEP,CONSTELLATION ENERGY PTNRS L L C,NaN,NaN,51957,91845,210380000000000,210380000000000,ConstellationEG,exclude,NaN,NaN,NaN,NaN,2014 coy and ticker name change -- replaced by $SPP,NaN,NaN
8,CGR,CLAUDE RESOURCES INC,NaN,NaN,44704,89951,18287310,18287310,ClaudeResource,exclude,NaN,NaN,NaN,NaN,dropped in 2013,NaN,NaN
9,CMLP,CRESTWOOD MIDSTREAM PARTNERS L P,NaN,NaN,52701,92255,22637210,22637210,none,exclude,NaN,NaN,NaN,NaN,merger in 2013,NaN,NaN


<br>
We can also replace missing values with '0' on our row counts variable.

In [374]:
merged['num_articles'] = merged['num_articles'].replace([None], ['0'])

### Save output

Finally, let's save the output of our article dataframe and our company counts dataframe in both native PANDAS ('pickled') format and in Excel format.

In [377]:
df_WSJ.to_pickle('WSJ_articles.pkl')
merged.to_pickle('WSJ_counts.pkl')
df_WSJ.to_excel('WSJ_articles.xls', sheet_name='Sheet1')
merged.to_excel('WSJ_counts.xls', sheet_name='Sheet1')